In [1]:
import numpy as np #For matrix operations
import matplotlib.pyplot as plt #For plotting
import mpmath as mp #For hypergeometric function
import scipy.special as ssp #For special functions like gamma, etc.
import qutip as qt #For Quantum Mechanical Calculations
from tabulate import tabulate #Import tool for creating table

In [2]:
#Defining value of useful parameters
mu = 5.17/2
hbar = 1
omega1 = 0.2021667
beta1 = np.sqrt((mu*omega1)/hbar)
d = (hbar**2)/(2*mu)

In [3]:
#Defining Combination function Using Gamma Function
def cb(lam, mu):
    return ssp.gamma(lam+1)/((ssp.gamma(lam+1 - mu))*(ssp.gamma(mu+1)))

In [4]:
#Defining Kronecker Delta
def kron_del(m, n):
    if m == n:
        return 1
    else:
        return 0

$$N_{k\ell} = \sqrt{\frac{2\beta^3\, k!}{\Gamma\left(k + \ell + \frac{3}{2}\right)}}$$

In [5]:
#Defining Function for Normalization Constant
def Nl(k, l):
    return np.sqrt((2*(beta1**3)*ssp.factorial(k))/(ssp.gamma(k + l + 1.5)))

Define
$$J_p(k', k, \ell) = \int_0^\infty y^{\ell + 0.5(1 + p)}e^{-y}L_{k'}^{\ell + 0.5}(y)L_{k}^{\ell + 0.5}(y) dy$$

Now

$$J_0(k', k, \ell) =  \frac{(k+\ell + 0.5)!}{k!}\delta_{k', k}$$

Thus,

$$ I_{0}{(k', k, \ell)} = \frac{N_{k'\ell} N_{k\ell}}{2 \beta^{3}} \frac{(k+\ell + 0.5)!}{k!}\delta_{k', k} = \delta_{k', k} $$

In [6]:
#Function for I0 - Just Kronecker Delta
def I0(kp, k, l):
    return kron_del(kp, k)

$$J_{-1}{(k', k, \ell)} = \binom{k' - 0.5}{k'}\binom{k - 0.5}{k} \Gamma(\ell + 1)\; {}_{3}F_{2}\left({\begin{matrix}\ell+1,\; -k',\; -k\\ 0.5 - k',\; 0.5 - k\end{matrix}};1\right)$$

And

$$ I_{-1}{(k', k, \ell)} = \frac{N_{k'\ell} N_{k\ell}}{2 \beta^{2}} J_{-1}{(k', k, \ell)} $$

In [7]:
#Function for $J_{-1}$
def Jm1(kp, k, l):
    J1 = cb(kp-0.5, kp)
    J2 = cb(k-0.5, k)
    J3 = ssp.gamma(l+1)
    a1 = l+1
    a2 = -kp
    a3 = -k
    b1 = 0.5-kp
    b2 = 0.5-k
    J4 =mp.hyp3f2(a1, a2, a3, b1, b2, 1)
    return J1*J2*J3*J4

In [8]:
#Function for $I_{-1}$
def Im1(kp, k, l):
    resm1 = ((Nl(kp, l)*Nl(k, l))/(2*(beta1**2)))*Jm1(kp, k, l)
    return resm1

$$J_{1}{(k', k, \ell)} = \binom{k' - 1.5}{k'}\binom{k - 1.5}{k} \Gamma(\ell + 2)\; {}_{3}F_{2}\left({\begin{matrix}\ell+2,\; -k',\; -k\\ 1.5 - k',\; 1.5 - k\end{matrix}};1\right)$$

And

$$ I_{1}{(k', k, \ell)} = \frac{N_{k'\ell} N_{k\ell}}{2 \beta^{4}} J_{1}{(k', k, \ell)} $$

In [9]:
#Function for $J_{1}$
def J1(kp, k, l):
    J1 = cb(kp-1.5, kp)
    J2 = cb(k-1.5, k)
    J3 = ssp.gamma(l+2)
    a1 = l+2
    a2 = -kp
    a3 = -k
    b1 = 1.5-kp
    b2 = 1.5-k
    J4 =mp.hyp3f2(a1, a2, a3, b1, b2, 1)
    return J1*J2*J3*J4

In [10]:
#Function for $I_{1}$
def I1(kp, k, l):
    res1 = ((Nl(kp, l)*Nl(k, l))/(2*(beta1**4)))*J1(kp, k, l)
    return res1

$$J_2(k', k, \ell) = \frac{(k'+\ell + 0.5)!}{k'!}\left[(2k + \ell + 1.5)\delta_{k', k} - (k+\ell+0.5) \delta_{k', k-1} - (k+1)\delta_{k', k+1}\right]$$

And

$$I_{2}{(k', k, \ell)} = \frac{N_{k'\ell} N_{k\ell}}{2 \beta^{5}} J_{1}{(k', k, \ell)}$$

In [11]:
#Function for $J_{2}$
def J2(kp, k, l):
    fc = (ssp.gamma(kp + l + 1.5))/ssp.gamma(kp+1)
    f1 = (2*k) + l + 1.5
    f2 = k + l + 0.5
    f3 = k + 1
    return fc*((f1*kron_del(kp, k)) - (f2*kron_del(kp, k-1)) - (f3*kron_del(kp, k+1)))  

In [12]:
#Function for $I_{2}$
def I2(kp, k, l):
    res2 = ((Nl(kp, l)*Nl(k, l))/(2*(beta1**5)))*J2(kp, k, l)
    return res2

In [13]:
#Defining Useful parameters of the Perturbation Potential
C0 = 9.6293
kappa = 0.52
a = 2.34
v1 = 9.3434167
C1 = C0 - v1

$$H_{k', k} = \left<k', \ell|H_p|k, \ell\right>$$

In [14]:
#Defining Function for Calculating $H_{k', k}$ denoted as called Hmn here
def Hmn(kp, k, l):
    Hmn0 = C1*I0(kp, k, l)
    Hmn1 = kappa*Im1(kp, k, l)
    Hmn2 = (I1(kp, k, l))/(a**2)
    Hmn3 = 0.5*mu*(omega1**2)*I2(kp, k, l)
    return Hmn0 - Hmn1 + Hmn2 - Hmn3

In [19]:
#Function for Energy Eigenvalues of Toy SHO model
def En1(k, l):
    return v1 + ((2*k) + l + 1.5)*hbar*omega1

$$G_{k', k} = \begin{cases}
0 \hspace{1.69cm} \text{if}\; k' = k \\
\frac{H_{k', k}^2}{E_{k, \ell} - E_{k', \ell}} \;\;\; \text{otherwise}
\end{cases}$$

In [20]:
#Defining Function for Calculating $G_{k', k}$ denoted as called Gmn here
def Gmn(kp, k, l):
    if kp == k:
        return 0
    else:
        g1 = Hmn(kp, k, l)
        g2 = En1(k, 0) - En1(kp, 0)
        return (g1**2)/g2

In [21]:
#Function for Calculating second order energy correction for nth state, by summer Gmn over M states
def sec_cor(n, M):
    en2 = 0
    for i in range(M):
        en2 = en2 + Gmn(i, n, 0)
    return en2

In [24]:
#Calculating the Energy corrections, and comparing the net result from actual energy eigenvalues
header = ["E_n^{(0)}", "E_n^{(1)}", "E_n^{(2)}", "E_n^{(0)} + E_n^{(1)} + E_n^{(2)}", "Actual Energies (E_n)"]

Col1 = np.array([9.64667341, 10.0510042,  10.45532851, 10.85964687])
Col2 = np.array([Hmn(i, i, 0) for i in range(4)])
Col3 = np.array([sec_cor(i, 160) for i in range(4)])

Col4 = Col1 + Col2 + Col3
Col5 = np.array([9.45999529, 10.05155864, 10.39672205, 10.67363567])

zipped_lists = zip(Col1, Col2, Col3, Col4, Col5)

#Printing List
print(tabulate(zipped_lists, header, tablefmt="fancy_grid"))

╒═════════════╤═════════════╤═════════════╤═════════════════════════════════════╤═════════════════════════╕
│   E_n^{(0)} │   E_n^{(1)} │   E_n^{(2)} │   E_n^{(0)} + E_n^{(1)} + E_n^{(2)} │   Actual Energies (E_n) │
╞═════════════╪═════════════╪═════════════╪═════════════════════════════════════╪═════════════════════════╡
│     9.64667 │ -0.00485388 │  -0.114301  │                             9.52752 │                  9.46   │
├─────────────┼─────────────┼─────────────┼─────────────────────────────────────┼─────────────────────────┤
│    10.051   │  0.00620564 │  -0.0199358 │                            10.0373  │                 10.0516 │
├─────────────┼─────────────┼─────────────┼─────────────────────────────────────┼─────────────────────────┤
│    10.4553  │ -0.0501805  │  -0.0152286 │                            10.3899  │                 10.3967 │
├─────────────┼─────────────┼─────────────┼─────────────────────────────────────┼─────────────────────────┤
│    10.8596  │ -0.13726    